In [59]:
import numpy as np
import cv2
from skimage.feature import hog, local_binary_pattern
from os import listdir, mkdir
from os.path import isfile, join, dirname, exists
from PIL import Image
import os
# from sklearn.svm import SVC
from scipy.signal import convolve2d
# from sklearn.externals import joblib
# import joblib
import matplotlib.pyplot as pyplot
import pickle
import time


In [60]:
# model=pickle.load("Logistic Regression.pkl")
with open('Logistic Regression.pkl', 'rb') as file:
    # Deserialize and load the object from the file
    model = pickle.load(file)

In [61]:

#rotate the image with given theta value
def rotate(img, theta):
    rows, cols = img.shape[0], img.shape[1]
    image_center = (cols/2, rows/2)
    
    M = cv2.getRotationMatrix2D(image_center,theta,1)

    abs_cos = abs(M[0,0])
    abs_sin = abs(M[0,1])

    bound_w = int(rows * abs_sin + cols * abs_cos)
    bound_h = int(rows * abs_cos + cols * abs_sin)

    M[0, 2] += bound_w/2 - image_center[0]
    M[1, 2] += bound_h/2 - image_center[1]

    # rotate orignal image to show transformation
    rotated = cv2.warpAffine(img,M,(bound_w,bound_h),borderValue=(int(img[0][0]),int(img[0][0]),int(img[0][0])))
    return rotated
def round_to_angle_multiples(number):

    # Define the possible angles
    angles = [45, -45, 90, -90, 135, -135]

    # Calculate the absolute difference between the number and each angle
    differences = [abs(number - angle) for angle in angles]

    # Find the index of the angle with the smallest difference
    min_index = differences.index(min(differences))

    # Return the corresponding angle
    return angles[min_index]


def rotate_img(edgeimage,originalimage):
    # Apply Hough Lines Transform
    lines = cv2.HoughLinesP(edgeimage, rho=1, theta=np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)
    
    finalImage=originalimage
    # Find the longest line
    longest_line = None
    max_length = 0
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Calculate line length using distance formula
            length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            if length > max_length:
                max_length = length
                longest_line = line

        if longest_line is not None:
            x1, y1, x2, y2 = longest_line[0]
            if x2 - x1 != 0:  # Avoid division by zero
                 orientation = np.arctan((y2 - y1) / (x2 - x1))*180  # Radians
            else:
                orientation = 180/2  # Vertical line (slope is undefined)
            # print("oo",orientation)
            if (orientation>20.0 or orientation<-20):
                orientation=round_to_angle_multiples(orientation)
                finalImage = rotate(originalimage, 180-orientation)
        
        # # print(orientation)
        # # Draw only the longest line (if any)
        # if longest_line is not None:
        #     x1, y1, x2, y2 = longest_line[0]
        #     cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 2)  # Draw blue line with thickness 2
        #     display(img,"box")

    return finalImage

In [62]:
def preprocess(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Salt and pepper noise
    img=cv2.medianBlur(img,5)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # display(img)

    # cv2.imshow('Detected Lines (Longest)', img)
    # Apply Canny edge detection
    edges = cv2.Canny(img, 20, 150)  # Adjust threshold values as needed

    rotated_img=rotate_img(edges,img)
    # display(img)
    if (img[0][0]>200):
        _, img = cv2.threshold(rotated_img, 50.0, 255.0, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    else:
        _, img = cv2.threshold(rotated_img, 50.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # display(img)
    return img

def process_image(image_path, output_dir):
    img = preprocess(image_path)  # Assuming preprocess returns a NumPy array
    img = Image.fromarray(img)  # Convert to PIL Image object
    new_path = join(output_dir, image_path.split("/")[-1])
    img.save(new_path)
    
def process_folder(folder_path):
    for folder in listdir(folder_path):
        img = join(folder_path, folder)
        if isfile(img) and img.lower().endswith(".jpeg"):
        # Check for image files with common extensions
            process_image(img, "Processed-fonts-dataset")    

In [63]:
def lpq(img_filepath, winSize=3,freqestim=1,mode='h'):

	img = cv2.imread(img_filepath,0)
	rho=0.90

	STFTalpha=1/winSize  # alpha in STFT approaches (for Gaussian derivative alpha=1)
	sigmaS=(winSize-1)/4 # Sigma for STFT Gaussian window (applied if freqestim==2)
	sigmaA=8/(winSize-1) # Sigma for Gaussian derivative quadrature filters (applied if freqestim==3)

	convmode='valid' # Compute descriptor responses only on part that have full neigborhood. Use 'same' if all pixels are included (extrapolates np.image with zeros).

	img=np.float64(img) # Convert np.image to double
	r=(winSize-1)/2 # Get radius from window size
	x=np.arange(-r,r+1)[np.newaxis] # Form spatial coordinates in window

	if freqestim==1:  #  STFT uniform window
	    #  Basic STFT filters
			w0=np.ones_like(x)
			w1=np.exp(-2*np.pi*x*STFTalpha*1j)
			w2=np.conj(w1)

	## Run filters to compute the frequency response in the four points. Store np.real and np.imaginary parts separately
	# Run first filters
	filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
	filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
	filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
	filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

	# Initilize frequency domain matrix for four frequency coordinates (np.real and np.imaginary parts for each frequency).
	freqResp=np.dstack([filterResp1.real, filterResp1.imag,
	                    filterResp2.real, filterResp2.imag,
	                    filterResp3.real, filterResp3.imag,
	                    filterResp4.real, filterResp4.imag])

	## Perform quantization and compute LPQ codewords
	inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
	LPQdesc=((freqResp>0)*(2**inds)).sum(2)

	## Histogram if needed
	if mode=='nh' or mode=='h':
	    LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

	## Normalize histogram if needed
	if mode=='nh':
	    LPQdesc=LPQdesc/LPQdesc.sum()

	# print (LPQdesc)
	LPQdesc = np.array(LPQdesc)
	return LPQdesc

In [64]:
# test_features=[]
# image_predictions={}
# image_time={}

# for folder in os.listdir("test"):
#     img = os.path.join("test", folder)
#     if isfile(img) and img.lower().endswith(".jpeg"):
#         start_time = time.time()
#         process_image(img,"Processed-test")
#         path="Processed-test/test/"+folder
#         # print(path)
#         imageLPQ = lpq(path)  # Assuming lpq is defined elsewhere
#         # print(imageLPQ)
#         label=model.predict(np.array([imageLPQ]))
#         end_time_prediction = time.time() 
#         image_predictions[int(folder[:-5])]=label
#         image_time[int(folder[:-5])]=round(end_time_prediction-start_time,3)
#         # print(model.predict(np.array([imageLPQ])),"\n")

#     # break
#         # print(f"Processed image: {img}")
#         # print("-----------------------------------")

# sorted_predictions = dict(sorted(image_predictions.items(), key=lambda item: item[0]))
# output_text = ""
# for img_name, prediction in sorted_predictions.items():
#   output_text += f"{prediction[0]}\n"

# with open("results.txt", "w") as output_file:
#   output_file.write(output_text)

# sorted_time = dict(sorted(image_time.items(), key=lambda item: item[0]))
# output_time = ""
# for img_name, time in sorted_time.items():
#   output_time += f"{time[0]}\n"

# with open("time.txt", "w") as output_file:
#   output_file.write(output_time)

# # for filename in os.listdir("Processed-test/test"):
# #     if filename.endswith(".jpeg"):
# #         image_path = os.path.join(filename, filename)
# #         print(image_path)

# # test_features=np.array(test_features)
# # print(test_features.shape)

# # labels=model.predict(test_features)
# # print(labels)
# # for i, label in enumerate(labels):
# #     print(f"Image {i + 1} has label: {label}")